#Introduction
  
Multiple linear Regression is to examine how multiple independent variables are related to a dependent variable.In this notebook i am concerned about Feature Selection. It is the important step which impact the model and its performance. if we put all the variable into the model even the best model will perform worst as Garbage In Garbage Out.So we need to select the most relevant features and drop the irrelevant features


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Importing the dataset

In [ ]:
dataset = pd.read_csv('/kaggle/input/startup-logistic-regression/50_Startups.csv')


In [ ]:
dataset.info()


 There are 50 instances/rows in the dataset which means that it is fairly small when compared to real world datasets in machine learning. Notice that there are 4 columns has datatype float64(Numnerical Attribute) and 1 column has obiect type(Categorical Attribute). Lets first check the numerical Attributes

In [ ]:
dataset.describe()

 This is the minimum, first quartile, median, third quartile, and maximum. So, the minimum is our smallest value. The first quartile often called the 25th percentile is where 25 percent of the data falls below that value. The median which is sometimes called the 50th percentile is where 50 percent falls below that value and the third quartile often called the 75th percentile is where 75 percent falls below that value and then finally, we have our maximum value


## A histogram for each numerical attribute


In [ ]:
#scatterplot
sns.set()
cols = ['Profit', 'R&D Spend', 'Marketing Spend', 'Administration']
sns.pairplot(dataset[cols], height = 2.5)
plt.show();

# Correlation Matrix


The correlation coefficient ranges from –1 to 1. When it is close to 1, it means that there is a strong positive correlation; for example,the Profit tends to go up when R&D Spend goes up. When the coefficient is close to –1, it means that there is a strong negative correlation;  you can see the Administration is less correlated than other with Profit. Finally, coefficients close to zero mean that there is no linear correlation.

In [ ]:
%matplotlib inline
plt.figure(figsize=(12,10))
cor = dataset.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()


In [ ]:
corr_Profit=cor["Profit"].sort_values(ascending=False)
print(corr_Profit)

# Checking for Outliers

An Outlier is the data that does not fit with other data in dataset ,like error while data correction or ture outliers i.e the data in out range.Outliers can be found using different methods:
* Box plot
* Z-Score
* IQRS Score

## Finding, Removing Outliers using IQR Score

 The interquartile range (IQR), also called the midspread or middle 50%, or technically H-spread, is a measure of statistical dispersion, being equal to the difference between 75th and 25th percentiles, or between upper and lower quartiles, IQR = Q3 − Q1.

In [ ]:
Q1 = dataset.quantile(0.25)
Q3 = dataset.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

The data printed below has two values False and True whereas True Indicates the Presence of the Outliers and False indicates the absence

In [ ]:
print(dataset < (Q1 - 1.5 * IQR)) |(dataset > (Q3 + 1.5 * IQR))

## Deleting the Outliers

In [ ]:
dataset_outl = dataset[~((dataset < (Q1 - 1.5 * IQR)) |(dataset > (Q3 + 1.5 * IQR))).any(axis=1)]
dataset_outl.shape

# Checking the target variable

In [ ]:
#Normality
#histogram and normal probability plot
from scipy.stats import norm
from scipy import stats
sns.distplot(dataset['Profit'], fit=norm);
fig = plt.figure()
res = stats.probplot(dataset['Profit'], plot=plt)

In [ ]:
# splitting the dataset into  independent variable X and dependent(target) y
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

## Preparing the Data for Modeling
 The input features which we pass to the model should contain only the numerical data which the model perfomance and accuracy, so we need to convert the categorical varaible state ito the Numerical attribure by using one-hot encoding and column transformer


In [ ]:
# Encoding categorical data
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
 
ct = ColumnTransformer([('encoder', OneHotEncoder(),[3])], remainder='passthrough')
 
X = np.array(ct.fit_transform(X), dtype=np.float)



## Avoiding the Dummy Variable Trap


In [ ]:
X = X[:, 1:]

# Feature Selection
 You need not use every feature in your model. You can feed only important feature,Now lets see how to figure out the important one. As we encoded our data in last step we got only numerical data and there are serveral methods for Numeric feature selection
There are many methods to do Feature selection:
* Filter Method
* wrapper methods
* Embedded Method

There are different wrapper methods: 
* Backward Elimination
* Forward Selection 
* Bidirectional Elimination 
* RFE.

Here I am applying the RFE for selecting the most correlated features with target varaible
RFE (Recursive Feature Elimination)
It then gives the ranking of all the variables, 1 being most important. It also gives its support, True being relevant feature and False being irrelevant feature.
Here we took LinearRegression model with 2 features and the selection of number ‘2’ was random,Lets find the ranking for features

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
# drop the 1 column in X
RFE_regressor = LinearRegression()
#Initializing RFE model
rfe = RFE(RFE_regressor, 2)# random number(2)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
RFE_regressor.fit(X,y)
print(rfe.support_)
print(rfe.ranking_)

 Now we need to find the optimum number of features, for which the accuracy is the highest. We do that by using loop starting with 1 feature and going up to 4. We then take the one for which the accuracy is highest.

In [ ]:
#-------before calc high score it says all true ----

from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LinearRegression
#no of features
nof_list=np.arange(1,4)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))


The optimum number of features is 3 .lets apply back into Linear Regression with 3 feature to get the ranking

In [ ]:
#Initializing RFE model
rfe = RFE(RFE_regressor, 3)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
RFE_regressor.fit(X,y)
print(rfe.support_)
print(rfe.ranking_)


In [ ]:
RFE_features=X[:,[0,1,2]] # using the features with only True values

# Fitting the model

In [ ]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
print("score: ",regressor.score(X_train,y_train))
print("Model slope:    ", regressor.coef_)
print("Model intercept:", regressor.intercept_)

# Predict

In [ ]:
# Predicting the Test set results
y_pred = regressor.predict(X_test)
print("score: ",regressor.score(X_test,y_test))
print("Model slope:    ", regressor.coef_)
print("Model intercept:", regressor.intercept_)